# Intro to Dash: Visualizing workout data with reactive plots

Dash is a low code web application framework for rapidly building data apps in Python. It is written on top of Plotly and React. It is a great tool for building data driven web apps.

In [5]:
import pandas as pd


"""Creates a dataframe of workouts from peloton workouts csv. Can be downloaded from 
https://members.onepeloton.com/profile/workouts
"""
df = pd.read_csv('workouts.csv')


## Step 1: Import our dependencies

Dash makes use of plotly for its charts. We are using dash-bootstrap-components here for our html components and styling. We are importing dash-core-components (dcc) for dash components such as dropdowns and graphs. The dash html components provide an easy way for us to create html components. jupyter dash is used for developing within jupyter notebooks. Finally, we are importing Input and Output and state from dash.dependencies. These are used for callbacks.

**Useful documentation links:**

- Plotly: [make_subplots](https://plotly.com/python-api-reference/generated/plotly.subplots.make_subplots.html)
- Dash Core Components: [dash.dcc](https://dash.plotly.com/dash-core-components)
- Dash HTML Components: [dash.html](https://dash.plotly.com/dash-html-components)
- Jupyter Dash: [jupyter dash](https://dash.plotly.com/jupyter-dash)

In [2]:
import plotly.graph_objects as go
import dash_bootstrap_components as dbc

from plotly.subplots import make_subplots
from dash import dcc, html
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State

## Step 2: Filter and group data

In [6]:
ts ='Workout Timestamp'
def group_data_by_timestamp(start_date = df[ts].min(), end_date = df[ts].max() , freq: str ='W') -> pd.DataFrame:
    """
    Filters dataframe by date range and groups by frequency of week by default.
    Returns a dataframe with Calories Burned and Total Output summed by the chosen frequency.

    Parameters
    ----------
        start_date : date The start date of the range
        end_date : date The end date of the range
        freq : str The frequency of the group. Default is weekly.

    Returns
    -------
        df : pd.DataFrame The dataframe with the summed columns.

    """
    dff = df.loc[(start_date <= df[ts]) & (df[ts] <= end_date)]

    for tz in ['EST', 'EDT', '-04', '-05']:
        dff.loc[:, ts]= dff[ts].str.replace(f"\({tz}\)", '', regex=True)

    dff.loc[:,ts] = pd.to_datetime(dff[ts], format='%Y-%m-%d %H:%M', errors='coerce')
    return dff.groupby(pd.Grouper(key=ts, freq=freq))[['Calories Burned', 'Total Output']].agg('sum').reset_index()

group_data_by_timestamp()

,Workout Timestamp,Calories Burned,Total Output
0,2020-12-20,21.0,0.0
1,2020-12-27,100.0,0.0
2,2021-01-03,151.0,0.0
3,2021-01-10,403.0,0.0
4,2021-01-17,100.0,0.0
...,...,...,...
58,2022-01-30,2615.0,670.0
59,2022-02-06,1402.0,439.0
60,2022-02-13,1346.0,454.0
61,2022-02-20,918.0,412.0


## Step 3: Create our plot

Now that we have our data grouped as we'd like to see it, lets plot it using plotly. Since Dash is built on top of plotly, we will be able to use this figure in our web app later.

In [ ]:
def create_timeseries_figure(df: pd.DataFrame, frequency = "W") -> go.Figure:
    """
    Creates a timeseries figure with two subplots. The first subplot graphs the Total Output for the week.
    The second subplot graphs the Calories Burned for the week.

    Parameters
    ----------
        df : pd.DataFrame The dataframe to be plotted.
        frequency : str The frequency of the group. Default is weekly.
    
    Returns
    -------
        go.Figure: The figure to be plotted.
    """
    switcher = {
        'D': 'Day',
        'W': 'Week',
        'M': 'Month'
    }
    freq = switcher.get(frequency, 'Week')
    line = make_subplots(specs=[[{"secondary_y": True}]])
    line.add_trace(go.Scatter(x=df['Workout Timestamp'], y=df['Calories Burned'],
                                marker=dict(size=10, color='MediumPurple'),
                                name='Total Calories'
                            ),
                            secondary_y=False
    )
    line.add_trace(go.Scatter(x=df['Workout Timestamp'], y=df['Total Output'],
                                marker=dict(size=10, color='MediumSeaGreen'),
                                name='Total Output'
                            ),
                    secondary_y=True
    )

    line.update_layout(
        title=f"Calories and Total Output per {freq}",
        title_x=0.5,
        yaxis_title="Calories Burned",
    )

    line.update_yaxes(title_text="Total Output", secondary_y=True)
    line.update_yaxes(title_text="Calories Burned", secondary_y=False)
    line.update_xaxes(title=f"{freq}")
    return line
create_timeseries_figure(group_data_by_timestamp())

## Step 4: Add our plot to our dashboard

We now have our data and our plot. We are ready to start building our dashboard. Below we will create the layout for the html to display the plot in.

In [ ]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

def create_calories_and_output_card(workout_df) -> dbc.Card:
    """
    Creates a dbc card with a title, DatePicker, RadioButton, and a timeseries figure.
    """
    return dbc.Card([
        dbc.CardBody([
            html.Center(html.H1("Weekly Calorie and Output Breakdown", className='card-title')),
            html.Center(
                dcc.DatePickerRange(
                    id='date-range-picker-2',
                    min_date_allowed=workout_df['Workout Timestamp'].min(),
                    max_date_allowed=workout_df['Workout Timestamp'].max(),
                    initial_visible_month=workout_df['Workout Timestamp'].min(),
                    start_date=workout_df['Workout Timestamp'].min(),
                    end_date=workout_df['Workout Timestamp'].max(),
                    style={
                        "margin-top": "1rem",
                        "margin-bottom": "1rem"
                    }
                )
            ),
            html.Center(dcc.RadioItems(
                options=[
                    {'label': 'Daily', 'value': 'D'},
                    {'label': 'Weekly', 'value': 'W'},
                    {'label': 'Monthly', 'value': 'M'},
                ],
                value='W',
                id='calories-radio-items',
                labelStyle={'padding-right': '20px'}
            ),
            ),
            dcc.Graph(
                id='weekly-calories-burned-chart'
            )
        ])
    ],
        outline=True,
        color='info',
        style={
            "margin-left": "1rem",
            "margin-bottom": "1rem"
        }
    )

@app.callback(
    Output('weekly-calories-burned-chart', 'figure'),
    [   Input('date-range-picker-2', 'start_date'),
        Input('date-range-picker-2', 'end_date'),
        Input('calories-radio-items', 'value')]
)
def update_weekly_calories_burned_chart(start_date, end_date, frequency):
    """
    Updates the weekly calories burned chart with the selected date range and frequency.
    """
    grouped_df = group_data_by_timestamp(start_date, end_date, frequency)
    return create_timeseries_figure(grouped_df, frequency)

app.layout = html.Div(
    children=[
        dbc.Row([create_calories_and_output_card(df)])
    ]
)

app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


c:\Users\jrami\VSCodeProjects\dash-presentation\venv\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

C:\Users\jrami\AppData\Local\Temp\ipykernel_30316\2243030622.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jrami\AppData\Local\Temp\ipykernel_30316\2243030622.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jrami\AppData\Local\Temp\ipykernel_30316\2243030622.py